考慮一個實際問題，某城市在 2013 年 - 2017 年的房價如下表所示：

年份

2013

2014

2015

2016

2017

房價

12000

14000

15000

16500

17500

現在，我們希望通過對該資料進行線性回歸，即使用線性模型 y = ax + b 來擬合上述資料，此處 a 和 b 是待求的參數。

首先，我們定義資料，進行基本的正規化操作。

In [1]:
import numpy as np
import tensorflow as tf

X_raw = np.array([2013, 2014, 2015, 2016, 2017], dtype=np.float32)
y_raw = np.array([12000, 14000, 15000, 16500, 17500], dtype=np.float32)

X = (X_raw - X_raw.min()) / (X_raw.max() - X_raw.min())
y = (y_raw - y_raw.min()) / (y_raw.max() - y_raw.min())

print(X)
print(y)

[0.   0.25 0.5  0.75 1.  ]
[0.         0.36363637 0.54545456 0.8181818  1.        ]


回顧機器學習的基礎知識，對於多元函數 求局部極小值，梯度下降 的過程如下：

初始化自變數為 x_0 ， k=0

疊代進行下列步驟直到滿足收斂條件：

求函数 f(x) 關於自變數的梯度 \nabla f(x_k)

更新自變數： x_{k+1} = x_{k} - \gamma \nabla f(x_k) 。這裡 \gamma 是學習率（也就是梯度下降一次邁出的 “步長” 大小）

k \leftarrow k+1

接下來，我們考慮如何使用程式來實現梯度下降方法，求得線性回歸的解 \min_{a, b} L(a, b) = \sum_{i=1}^n(ax_i + b - y_i)^2 

**NumPy 下的線性回歸**

In [2]:
a, b = 0, 0

num_epoch = 10000
learning_rate = 5e-4
for e in range(num_epoch):
    # 手動計算損失函數關於自變數（模型參數）的梯度
    y_pred = a * X + b
    grad_a, grad_b = 2 * (y_pred - y).dot(X), 2 * (y_pred - y).sum()

    # 更新參數
    a, b = a - learning_rate * grad_a, b - learning_rate * grad_b

print(a, b)

0.9763702027872221 0.057564988311377796


**TensorFlow 下的線性回歸**

TensorFlow 的 即時執行模式 5 與上述 NumPy 的運行方式十分類似，然而提供了更快速的運算（GPU 支援）、自動推導、優化器等一系列對深度學習非常重要的功能。以下展示了如何使用 TensorFlow 計算線性回歸。可以注意到，程式的結構和前述 NumPy 的實現非常類似。這裡，TensorFlow 幫助我們做了兩件重要的工作：

使用 tape.gradient(ys, xs) 自動計算梯度；

使用 optimizer.apply_gradients(grads_and_vars) 自動更新模型參數

In [3]:
X = tf.constant(X)
y = tf.constant(y)

a = tf.Variable(initial_value=0.)
b = tf.Variable(initial_value=0.)
variables = [a, b]

num_epoch = 10000
optimizer = tf.keras.optimizers.SGD(learning_rate=5e-4)
for e in range(num_epoch):
    # 使用tf.GradientTape()記錄損失函數的梯度資訊
    with tf.GradientTape() as tape:
        y_pred = a * X + b
        loss = tf.reduce_sum(tf.square(y_pred - y))
    # TensorFlow自動計算損失函數關於自變數（模型參數）的梯度
    grads = tape.gradient(loss, variables)
    # TensorFlow自動根據梯度更新參數
    optimizer.apply_gradients(grads_and_vars=zip(grads, variables))

print(a, b)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.97637> <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.057565063>
